#Section 1. - Importing and preprocessing the data

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Loading the dataset
url = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/delaney-processed.csv"
df = pd.read_csv(url)

# Converting SMILES to RDKit Mol objects
df['mol'] = df['smiles'].apply(Chem.MolFromSmiles)

# Calculating RDKit descriptors
def calculate_descriptors(mol):
    descriptors = {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol)
    }
    return pd.Series(descriptors)

df = df.join(df['mol'].apply(calculate_descriptors))

# Dropping rows with missing values (where molecules couldn't be processed)
df = df.dropna()

# Defining features (X) and target (y)
X = df[['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'Number of Rings', 'Number of Rotatable Bonds', 'Polar Surface Area']]
y = df['measured log solubility in mols per litre']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Section 2 - Feeding ML algorithms with data

#Section 2.1 - Linear Regression

In [3]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression()
linear.fit(X_train, y_train)

# Making predictions
y_pred = linear.predict(X_test)

# Evaluating the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')  

RMSE: 1.0728493823684075
R² Score: 0.7564938335582524


c:\Users\Admin\Documents\Coding\Cheminf\RdKit_grind\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#Section 2.2 - Random Forest Regression

In [4]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=100, criterion='squared_error', max_depth=None)
forest.fit(X_train, y_train)

# Making predictions
y_pred = forest.predict(X_test)

# Evaluating the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')  

RMSE: 0.8135865259486826
R² Score: 0.8599639192368479


c:\Users\Admin\Documents\Coding\Cheminf\RdKit_grind\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#Section 2.3 - Support Vector Machines

In [7]:
from sklearn.svm import SVR

# Define the SVR model with RBF kernel
svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# Train the model
svr.fit(X_train_scaled, y_train)

# Make predictions
y_pred = svr.predict(X_test_scaled)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')

RMSE: 0.8780779264809295
R² Score: 0.8368832453712077


c:\Users\Admin\Documents\Coding\Cheminf\RdKit_grind\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#Section 2.4 - Multi Layer Perceotron

In [6]:
# Scaling the data (important for MLPs)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Defining and train the MLPRegressor model
mlp = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Making predictions
y_pred = mlp.predict(X_test_scaled)

# Evaluating the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')    

RMSE: 0.7289858385642586
R² Score: 0.8875729964215217


c:\Users\Admin\Documents\Coding\Cheminf\RdKit_grind\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#Section 2.5 - Convolutional Neural Networks

In [9]:
from rdkit.Chem import AllChem
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Load the dataset
url = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/delaney-processed.csv"
df = pd.read_csv(url)

# Convert SMILES to RDKit Mol objects
df['mol'] = df['smiles'].apply(Chem.MolFromSmiles)

# Convert molecules to molecular fingerprints
def mol_to_fingerprint(mol, radius=2, n_bits=2048):
    """Convert a molecule to a Morgan fingerprint."""
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)

# Calculate fingerprints and convert them to numpy arrays
df['fingerprint'] = df['mol'].apply(mol_to_fingerprint)
X = pd.DataFrame(df['fingerprint'].apply(lambda x: list(x)).tolist())
y = df['measured log solubility in mols per litre']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to numpy arrays and reshape for 1D convolution (CNN expects 3D input: samples, timesteps, features)
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=8, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Prevent overfitting
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1))  # Regression task, output single value (solubility)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Make predictions
y_pred = model.predict(X_test).flatten()

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')


[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerator
[23:31:35] DEPRECATION WARNING: please use MorganGenerat

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 9s 219ms/step - loss: 6.0609 - val_loss: 2.2312
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 199ms/step - loss: 1.7343 - val_loss: 1.8746
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 212ms/step - loss: 1.0892 - val_loss: 1.7127
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 220ms/step - loss: 0.8769 - val_loss: 1.6954
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 219ms/step - loss: 0.8068 - val_loss: 1.7346
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 206ms/step - loss: 0.5901 - val_loss: 1.6428
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 206ms/step - loss: 0.4717 - val_loss: 1.6246
Epoch 8/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 212ms/step - loss: 0.3507 - val_loss: 1.6848
Epoch 9/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 206ms/step - loss: 0.3821 - val_loss: 1.6172
Epoch 10/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 207ms/step - loss: 0.3674 - val_loss: 1.6531
Epoch 11/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 203ms/step - loss: 0.3705 - val_loss: 1.6796
Epoch 12/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 199ms/step

c:\Users\Admin\Documents\Coding\Cheminf\RdKit_grind\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#Section 3 - Predictions based on SMILE format

In [11]:
from sklearn.preprocessing import StandardScaler

# Calculating descriptors for molecules in SMILE format
def prepare_raw_data(dataFrame):

    dataFrame['mol'] = dataFrame['smiles'].apply(Chem.MolFromSmiles)
    dataFrame = dataFrame.join(dataFrame['mol'].apply(calculate_descriptors))
    dataFrame = dataFrame.dropna()
    '''
    for molecule in dataFrame['mol']
        ring_info = molecule.GetRingInfo()
        num_rings = ring_info.NumRings()
        
    '''    
    return dataFrame

# Normalizng the data via StandardScaler
def normalize_data(dataframe):
    scaler = StandardScaler()
    normalized = scaler.fit_transform(dataframe)

    return dataframe

SECTION XX: Testing the ML model on data generated by using rdkit


In [12]:
smilesList = [
    'C', 'CC', 'CCC', 'CCCC', 'CCCCC', 'CCCCCC', 
    'CCCCCCC', 'CCN', 'CC(=O)O', 'CCOCC', 'CC(=O)CC', 
    'CC(=O)OCC', 'CCCCO', 'C(C(=O)O)C(N)C', 'CCCCCO'
]

# The solubities in a form of log([mol of substance]/[litre of H2O]). Experimental measurements based on the literature. 
log_solubilities = [-2.82, -2.70,	-2.82,	-2.98,	-2.65,	-3.96,	-4.53,	-1.57,	1.00,	-0.12,	0.44,	-0.10,	-0.01,	0.79,	-0.61]
# For refernece - "Yalkowsky, S.H., He, Yan, Jain, P. Handbook of Aqueous Solubility Data Second Edition. CRC Press, Boca Raton, FL 2010"


In [14]:
smilesDF = pd.DataFrame(smilesList, columns=['smiles'])

testDFfromSMILES = prepare_raw_data(smilesDF)

scaler = StandardScaler()

norm_testDFfromSMILES = normalize_data(testDFfromSMILES[['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors']])


In [15]:
predictions_MLP = mlp.predict(norm_testDFfromSMILES)
predictionsDF = norm_testDFfromSMILES.join(pd.DataFrame(predictions_MLP, columns=['MLP predictions']))

c:\Users\Admin\Documents\Coding\Cheminf\RdKit_grind\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


ValueError: X has 4 features, but MLPRegressor is expecting 7 features as input.

END
    

In [8]:
smilesDF

,smiles,mol
0,C,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
1,CC,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
2,CCC,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
3,CCCC,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
4,CCCCC,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
5,CCCCCC,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
6,CCCCCCC,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
7,CCN,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
8,CC(=O)O,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
9,CCOCC,<rdkit.Chem.rdchem.Mol object at 0x0000021879E...
